<a href="https://colab.research.google.com/github/tanuja1708/EEG-emotions/blob/main/emotion_prediction(GAN%2CLSTM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q tensorflow numpy pandas matplotlib seaborn scikeras


In [ ]:
import pandas as pd

df = pd.read_csv('/content/emotions.csv')
df.head()


,# mean_0_a,mean_1_a,mean_2_a,mean_3_a,mean_4_a,mean_d_0_a,mean_d_1_a,mean_d_2_a,mean_d_3_a,mean_d_4_a,...,fft_741_b,fft_742_b,fft_743_b,fft_744_b,fft_745_b,fft_746_b,fft_747_b,fft_748_b,fft_749_b,label
0,4.62,30.3,-356.0,15.6,26.3,1.070,0.411,-15.70,2.06,3.15,...,23.5,20.3,20.3,23.5,-215.0,280.00,-162.00,-162.00,280.00,NEGATIVE
1,28.80,33.1,32.0,25.8,22.8,6.550,1.680,2.88,3.83,-4.82,...,-23.3,-21.8,-21.8,-23.3,182.0,2.57,-31.60,-31.60,2.57,NEUTRAL
2,8.90,29.4,-416.0,16.7,23.7,79.900,3.360,90.20,89.90,2.03,...,462.0,-233.0,-233.0,462.0,-267.0,281.00,-148.00,-148.00,281.00,POSITIVE
3,14.90,31.6,-143.0,19.8,24.3,-0.584,-0.284,8.82,2.30,-1.97,...,299.0,-243.0,-243.0,299.0,132.0,-12.40,9.53,9.53,-12.40,POSITIVE
4,28.30,31.3,45.2,27.3,24.5,34.800,-5.790,3.06,41.40,5.52,...,12.0,38.1,38.1,12.0,119.0,-17.60,23.90,23.90,-17.60,NEUTRAL


In [ ]:
df.isnull().sum()


,0
# mean_0_a,0
mean_1_a,0
mean_2_a,0
mean_3_a,0
mean_4_a,0
...,...
fft_746_b,0
fft_747_b,0
fft_748_b,0
fft_749_b,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# Encode labels
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# Split into features and labels
X = df.drop('label', axis=1)
Y = df['label']

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=111)


In [ ]:
X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))


In [ ]:
from tensorflow.keras.layers import LSTM, Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# LSTM model
i_lstm = Input(shape=(X_train.shape[1], 1))
x_lstm = LSTM(256, return_sequences=True)(i_lstm)
x_lstm = Flatten()(x_lstm)
y_lstm = Dense(3, activation='softmax')(x_lstm)

model_lstm = Model(i_lstm, y_lstm)

# Compile LSTM
model_lstm.compile(optimizer=Adam(learning_rate=0.001),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

model_lstm.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 2548, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 2548, 256)           │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 652288)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │       1,956,867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,221,059 (8.47 MB)

 Trainable params: 2,221,059 (8.47 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model_lstm.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 139ms/step - accuracy: 0.7106 - loss: 11.5719 - val_accuracy: 0.9064 - val_loss: 5.4044
Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 117ms/step - accuracy: 0.9277 - loss: 3.1659 - val_accuracy: 0.9231 - val_loss: 2.6125
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 123ms/step - accuracy: 0.9497 - loss: 1.6283 - val_accuracy: 0.9599 - val_loss: 1.2801
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 115ms/step - accuracy: 0.9847 - loss: 0.1800 - val_accuracy: 0.9532 - val_loss: 1.1406
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - accuracy: 0.9760 - loss: 0.2404 - val_accuracy: 0.9699 - val_loss: 0.5154
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - accuracy: 0.9556 - loss: 0.8536 - val_accuracy: 0.9799 - val_loss: 0.3412
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - accuracy: 0.9787 - loss: 0.3386 - val_accuracy: 0.9900 - val_loss: 0.1087
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 123ms/step - accuracy: 0.9892 - loss: 0.0871 - val_accuracy: 0

In [ ]:
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.models import Sequential

def build_generator(input_dim, output_dim):
    model = Sequential([
        Dense(128, input_dim=input_dim),
        LeakyReLU(alpha=0.2),
        Dense(256),
        LeakyReLU(alpha=0.2),
        Dense(output_dim, activation='tanh')  # Output synthetic features
    ])
    return model

generator = build_generator(input_dim=100, output_dim=X_train.shape[1])
generator.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                      │ (None, 128)                 │          12,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 256)                 │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2548)                │         654,836 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 700,788 (2.67 MB)

 Trainable params: 700,788 (2.67 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.layers import Dropout

def build_discriminator(input_dim):
    model = Sequential([
        Dense(256, input_dim=input_dim),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(128),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1, activation='sigmoid')  # Binary classification (Real/Fake)
    ])
    model.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy', metrics=['accuracy'])
    return model

discriminator = build_discriminator(input_dim=X_train.shape[1])
discriminator.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 256)                 │         652,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 685,569 (2.62 MB)

 Trainable params: 685,569 (2.62 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.models import Model

# Freeze discriminator while training GAN
discriminator.trainable = False

gan_input = Input(shape=(100,))
fake_data = generator(gan_input)
gan_output = discriminator(fake_data)

gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(learning_rate=0.0002), loss='binary_crossentropy')

gan.summary()


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential (Sequential)              │ (None, 2548)                │         700,788 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_1 (Sequential)            │ (None, 1)                   │         685,569 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,386,357 (5.29 MB)

 Trainable params: 700,788 (2.67 MB)

 Non-trainable params: 685,569 (2.62 MB)

In [ ]:
import tensorflow as tf

def train_gan(epochs=1000, batch_size=32):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Train discriminator
        real_samples = X_train[np.random.randint(0, X_train.shape[0], half_batch)]
        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_samples = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_samples, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(fake_samples, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Print progress
        if epoch % 1000 == 0:
            print(f"Epoch {epoch}: [D loss: {d_loss[0]}] [G loss: {g_loss}]")

train_gan(epochs=5000, batch_size=64)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step

In [21]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Generate 1000 synthetic samples
noise = np.random.normal(0, 1, (1000, 100))
synthetic_data = generator.predict(noise)

# Ensure synthetic data matches LSTM input shape (samples, timesteps, features)
synthetic_data = synthetic_data.reshape(1000, X_train.shape[1], X_train.shape[2])

# Combine real and synthetic data
X_train_augmented = np.vstack((X_train, synthetic_data))

# Generate synthetic labels & ensure correct shape
num_classes = len(np.unique(Y_train))
synthetic_labels = np.random.randint(0, num_classes, 1000)

# Combine real and synthetic labels
Y_train_augmented = np.hstack((Y_train, synthetic_labels))

# Convert labels to categorical (if using classification)
Y_train_augmented = to_categorical(Y_train_augmented, num_classes=num_classes)

# Ensure dtype consistency
X_train_augmented = np.array(X_train_augmented, dtype=np.float32)
Y_train_augmented = np.array(Y_train_augmented, dtype=np.float32)

# Debugging output
print("X_train_augmented shape:", X_train_augmented.shape)  # Should be (num_samples, timesteps, features)
print("Y_train_augmented shape:", Y_train_augmented.shape)  # Should be (num_samples, num_classes)

# Train the LSTM model


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
X_train_augmented shape: (2492, 2548, 1)
Y_train_augmented shape: (2492, 3)


In [16]:
print(X_train_augmented.shape)  # Should be (num_samples, timesteps, features)
print(Y_train_augmented.shape)  # Should be (num_samples, output_dim)


(2492, 2548)
(2492,)


In [22]:
model_lstm.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [23]:
model_lstm.fit(X_train_augmented, Y_train_augmented, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - accuracy: 0.8222 - loss: 1.0513 - val_accuracy: 0.3327 - val_loss: 1.2164
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 115ms/step - accuracy: 0.8395 - loss: 0.4668 - val_accuracy: 0.3006 - val_loss: 1.4935
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 11s 127ms/step - accuracy: 0.8448 - loss: 0.4178 - val_accuracy: 0.3086 - val_loss: 1.1784
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 128ms/step - accuracy: 0.8463 - loss: 0.3795 - val_accuracy: 0.3206 - val_loss: 1.2506
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 117ms/step - accuracy: 0.8421 - loss: 0.4437 - val_accuracy: 0.3667 - val_loss: 1.2197
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 118ms/step - accuracy: 0.8354 - loss: 1.0253 - val_accuracy: 0.3186 - val_loss: 1.2863
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - accuracy: 0.8370 - loss: 0.9977 - val_accuracy: 0.2906 - val_loss: 1.4631
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 118ms/step - accuracy: 0.8560 - loss: 0.4076 - val_accuracy

In [26]:
from tensorflow.keras.utils import to_categorical

Y_test_one_hot = to_categorical(Y_test, num_classes=3)  # Adjust num_classes based on your dataset
loss, acc = model_lstm.evaluate(X_test, Y_test_one_hot)
print(f"Final Test Accuracy: {acc * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.9649 - loss: 2.7231
Final Test Accuracy: 96.88%


In [27]:
model_lstm.save('/content/drive/MyDrive/lstm_model.h5')  # Save to Google Drive


In [28]:
model_lstm.save('/content/drive/MyDrive/lstm_model.keras')  # Recommended format


In [29]:
generator.save('/content/drive/MyDrive/gan_generator.h5')


In [30]:
discriminator.save('/content/drive/MyDrive/gan_discriminator.h5')


In [33]:
from tensorflow.keras.models import load_model

# Load the trained GAN generator model
generator = load_model("/content/lstm_model_after GAN.h5")


In [35]:
print(generator.input_shape)  # Should show (None, 2548, 1)


(None, 2548, 1)


In [36]:
import numpy as np

# Generate a valid random input matching the model's expected shape
valid_input = np.random.randn(1, 2548, 1)  # Batch size of 1

# Predict using the trained generator
generated_sample = generator.predict(valid_input)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step


In [38]:
print(generated_sample.shape)


(1, 3)


In [39]:
print(generated_sample)


[[7.9271320e-04 4.4768313e-03 9.9473047e-01]]


In [43]:
generator.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 2548, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 2548, 256)           │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 652288)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 3)                   │       1,956,867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,221,061 (8.47 MB)

 Trainable params: 2,221,059 (8.47 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [54]:
import numpy as np
from keras.models import load_model

# Load the trained LSTM model
lstm_model = load_model("/content/lstm_model_after GAN.h5")  # Update with correct path

# Define emotion labels
emotion_labels = ["Positive", "Neutral", "Negative"]

# Generate or load a valid input (Feature Vector) for LSTM
valid_input = np.random.randn(1, 2548, 1)  # Ensure correct shape
print(valid_input)

# Predict using the LSTM model
emotion_prediction = lstm_model.predict(valid_input)  # Output: (1, 3)
print("Raw LSTM Model Output:", emotion_prediction)


# Get the index with the highest probability
predicted_index = np.argmax(emotion_prediction)

# Map the index to an emotion label
predicted_emotion = emotion_labels[predicted_index]

print(f"Predicted Emotion: {predicted_emotion}")


[[[-0.49829575]
  [ 2.02544254]
  [-0.51359328]
  ...
  [ 1.16068957]
  [-0.30223143]
  [-0.24033363]]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Raw LSTM Model Output: [[0.01548245 0.85878056 0.125737  ]]
Predicted Emotion: Neutral
